In [1]:
import geopandas as gpd
import os
from rasterstats import zonal_stats
import numpy as np
import tkinter as tk
from tkinter import filedialog, messagebox
import gdal
import rasterio
import ogr
import warnings
import json
import pandas as pd
from earthpy import clip
from shapely.geometry import JOIN_STYLE

import scipy.spatial
from pathlib import Path
warnings.filterwarnings('ignore')

root = tk.Tk()
root.withdraw()
root.attributes("-topmost", True)

''

Select Datasets

In [2]:
messagebox.showinfo('OnSSET extraction', 'Output folder')
workspace = filedialog.askdirectory()

messagebox.showinfo('OnSSET', 'Select the population map')
filename_pop = filedialog.askopenfilename(filetypes = (("rasters","*.tif"),("all files","*.*")))
pop=gdal.Open(filename_pop)
poprasterio=rasterio.open(filename_pop)

messagebox.showinfo('OnSSET', 'Select the nightlights map')
filename_NTL = filedialog.askopenfilename(filetypes = (("rasters","*.tif"),("all files","*.*")))
NTL = gdal.Open(filename_NTL)
NTLrasterio = rasterio.open(filename_NTL)

messagebox.showinfo('OnSSET', 'Select the admin map')
filename_admin = (filedialog.askopenfilename(filetypes = (("shapefile","*.shp"),("all files","*.*"))))
admin=gpd.read_file(filename_admin)

country_name = "Gambia"

Reclassify raster

In [3]:
kwargs2 = {'xRes': NTLrasterio.transform[0], 'yRes': NTLrasterio.transform[4], 'dstNodata':0} 

warped2 = gdal.Warp(workspace + r"/clippedNTL.tif", NTLrasterio.name, cutlineDSName=filename_admin, **kwargs2)

def reclassifyRasters(file):
    driver = gdal.GetDriverByName("GTiff")
    file = file
    band = file.GetRasterBand(1)
    lista = band.ReadAsArray()

    for j in range(file.RasterXSize):
        for i in range(file.RasterYSize):
            if lista[i,j] < 999999999 and lista[i,j] > 0:
                lista[i,j] = 1


    if file == pop:
        file2 = driver.Create(workspace + r"/reclassifiedPop.tif", file.RasterXSize , file.RasterYSize , 1)
    else:
        file2 = driver.Create(workspace + r"/reclassifiedNTL.tif", file.RasterXSize , file.RasterYSize , 1)
        
    file2.GetRasterBand(1).WriteArray(lista)

    proj = file.GetProjection()
    georef = file.GetGeoTransform()
    file2.SetProjection(proj)
    file2.SetGeoTransform(georef)
    file2.FlushCache()
    
NTL = gdal.Open(workspace + r"/clippedNTL.tif")  
reclassifyRasters(pop)
reclassifyRasters(NTL)

In [4]:
kwargs1 = {'xRes': 0.00083333, 'yRes': 0.00083333, 'dstNodata':0}
kwargs3 = {'xRes': NTLrasterio.transform[0], 'yRes': NTLrasterio.transform[4], 'dstNodata':0}

warped1 = gdal.Warp(workspace + r"/resampledPop.tif", workspace + r"/reclassifiedPop.tif", cutlineDSName=filename_admin, **kwargs1)
warped3 = gdal.Warp(workspace + r"/NTLArea.tif", workspace + r"/reclassifiedNTL.tif", cutlineDSName=filename_admin, **kwargs3)

In [5]:
def toPolygon(source, opt):
    sourceRaster = gdal.Open(source)
    band = sourceRaster.GetRasterBand(1)
    bandArray = band.ReadAsArray()
    
    if opt == 1:
        outShapefile = workspace + r"\clusters"
    else:
        outShapefile = workspace + r"\NTLArea"
        
    driver = ogr.GetDriverByName("ESRI Shapefile")


    if os.path.exists(outShapefile+".shp"):
        driver.DeleteDataSource(outShapefile+".shp")

    outDatasource = driver.CreateDataSource(outShapefile+ ".shp")

    outLayer = outDatasource.CreateLayer(outShapefile+ ".shp", srs=None)
    newField = ogr.FieldDefn('PLACEHOLDER', ogr.OFTInteger)
    outLayer.CreateField(newField)
    gdal.Polygonize( band, None, outLayer, 0, [], callback=None )
    outDatasource.Destroy()
    sourceRaster = None

toPolygon(workspace + r"/resampledPop.tif", 1)
toPolygon(workspace + r"/NTLArea.tif", 2)

NTLArea=gpd.read_file(workspace + r"/NTLArea.shp")
clean = NTLArea[NTLArea.PLACEHOLDE != 0]
clean_b = clean.buffer(0)
clean_b.to_file(workspace + r"/NTLArea.shp")

kwargs4 = {'xRes': poprasterio.transform[0], 'yRes': poprasterio.transform[4], 'dstNodata':0}
warped4 = gdal.Warp(workspace + r"/ElecPop.tif", poprasterio.name, cutlineDSName= workspace + r"/NTLArea.shp", **kwargs4)

elecPop=rasterio.open(workspace + r"/ElecPop.tif")
NTL = rasterio.open(workspace + r"/clippedNTL.tif")

In [6]:
clusters=gpd.read_file(workspace + r"\clusters.shp")
clean = clusters[clusters.PLACEHOLDE != 0]
buffered = clean.buffer(0.001)
convex_hull = buffered.convex_hull
buffered.to_file(workspace + "/clusters.shp")
convex_hull = gpd.read_file(workspace + r"/clusters.shp")
convex_hull["fid"] = 1
dissolved=convex_hull.dissolve(by="fid")

In [7]:
def multi2single(gpdf):
    gpdf_singlepoly = gpdf[gpdf.geometry.type == 'Polygon']
    gpdf_multipoly = gpdf[gpdf.geometry.type == 'MultiPolygon']

    for i, row in gpdf_multipoly.iterrows():
        Series_geometries = pd.Series(row.geometry)
        df = pd.concat([gpd.GeoDataFrame(row, crs=gpdf_multipoly.crs).T]*len(Series_geometries), ignore_index=True)
        df['geometry']  = Series_geometries
        gpdf_singlepoly = pd.concat([gpdf_singlepoly, df])

    gpdf_singlepoly.reset_index(inplace=True, drop=True)
    return gpdf_singlepoly

single = multi2single(dissolved)
single.to_file(workspace + r"\clusters.shp")

In [8]:
eps = 0.001
dissolved = single.buffer(eps, 1, join_style=JOIN_STYLE.mitre).buffer(-eps, 1, join_style=JOIN_STYLE.mitre)
dissolved.to_file(workspace + r"\clusters.shp")
dissolved = gpd.read_file(workspace + r"\clusters.shp")

admin = gpd.read_file(filename_admin)
lines = admin.boundary
dissolved_clip = clip.clip_shp(dissolved,admin)
buffered_lines = lines.buffer(0.000000001)
buffered_lines.to_file(workspace + r"\bufferedlines.shp")
buffered_lines = gpd.read_file(workspace + r"\bufferedlines.shp")

z = gpd.overlay(dissolved_clip, buffered_lines, how='difference')
z = z["geometry"] 
z.to_file(workspace + r"\clusters.shp")
z = gpd.read_file(workspace + r"\clusters.shp")
z['id'] = np.arange(len(z))

z.crs = {'init' :'epsg:4326'}
z_proj = z.to_crs({ 'init': 'epsg:3395'})
z_proj["GridCellAr"] = z_proj.area/1000000
z_proj["Country"] = country_name
z = z_proj.to_crs({ 'init': 'epsg:4326'})

In [9]:
clusters = zonal_stats(
    z,
    poprasterio.name,
    stats=["sum"],
    prefix="pop", geojson_out=True, all_touched=True)

clusters = zonal_stats(
    clusters,
    NTL.name,
    stats=["max"],
    prefix="NTL", geojson_out=True, all_touched=True)

clusters = zonal_stats(
    clusters,
    elecPop.name,
    stats=["sum"],
    prefix="ElecPop", geojson_out=True, all_touched=True)

In [10]:
output = workspace + r'\placeholder.geojson'
with open(output, "w") as dst:
    collection = {
        "type": "FeatureCollection",
        "features": list(clusters)}
    dst.write(json.dumps(collection))

clusters = gpd.read_file(output)
clusters.fillna(0, inplace=True)
os.remove(output)
clusters.to_file(workspace + r"\clusters.shp")